| Chapter  | Colab   | Kaggle          | Gradient      | Studio Lab             |
|:---------|:--------|:----------------|:--------------|:-----------------------|
| [Chapter 3: LIME for Transformers](3_model_explainability_and_interpretability/Chapter_3_LIME_for_Transformers.ipynb)                          | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/3_model_explainability_and_interpretability/Chapter_3_LIME_for_Transformers.ipynb)        | [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/3_model_explainability_and_interpretability/Chapter_3_LIME_for_Transformers.ipynb)        | [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/3_model_explainability_and_interpretability/Chapter_3_LIME_for_Transformers.ipynb)        | [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/3_model_explainability_and_interpretability/Chapter_3_LIME_for_Transformers.ipynb)        |
